In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 
from datetime import datetime
from statistics import NormalDist

# Plot volatility surface (price,strike,tenor) given an option (underlying,expiry)
qb = QuantBook()
ticker = "SPY"
tenyearrate = 0.047
# Define the trading day for the option prices
tradingday=datetime(2023, 9, 5)
nextday=tradingday + timedelta(days=1)

equity_symbol = qb.add_equity(ticker, data_normalization_mode=DataNormalizationMode.RAW).symbol
option = qb.add_option(equity_symbol)
# Price of underlying stock
atm=qb.history(TradeBar, equity_symbol, tradingday, nextday,Resolution.DAILY).close.values[0]
print(atm)
# Set filter options (minstrikes from currentprice,maxstrikes from currentprice,min expiry, max expiry)
#option.set_filter(-2,2,7,30)
# Set pricing model (see https://www.quantconnect.com/docs/v2/writing-algorithms/reality-modeling/options-models/pricing)
#option.price_model = OptionPriceModels.CrankNicolsonFD()

# Get list of contracts at start date
contract_symbols = qb.option_chain_provider.get_option_contract_list(equity_symbol, tradingday)
#for i in range(0,15):
#    print(contract_symbols[i], contract_symbols[i].id.strike_price)


In [ ]:


#Filter list according to requirements

#Set filtering % distance from at-the-money price
dist=0.5

strk_high = atm+(dist*atm)
strk_low = atm-(dist*atm)
# Check options by DTE
dTE=30
expy = tradingday+timedelta(dTE)
# call = 1 put = 0
option_type=1

filtered_contract_symbols=[]
for s in contract_symbols:
    if (s.id.option_right == option_type
        and strk_low <= s.id.strike_price <= strk_high
        and s.id.date <= expy
        ):
            filtered_contract_symbols.append(s)

# sort filtered contracts list
filtered_contract_symbols.sort(key=lambda x: (x.id.date,x.id.strike_price))


for i in range(0,10):
    print(filtered_contract_symbols[i].id.strike_price,
    "Call" if filtered_contract_symbols[i].id.option_right==1 else "Put",
    filtered_contract_symbols[i].id.date)


In [ ]:
contract_by_expiry={}
prehistory = datetime(1900,1,1)
expiry=prehistory
newcontractlist=[]
for s in filtered_contract_symbols:
    if expiry != s.id.date:
        contract_by_expiry.update({expiry:newcontractlist})
        newcontractlist=[]
        expiry = s.id.date
    newcontractlist.append(s)

contract_by_expiry.pop(prehistory)
for x in contract_by_expiry.keys():
    print(contract_by_expiry.get(x)[0].id,"Expiry ", x.strftime('%d/%m/%Y'),"Strike Price ", contract_by_expiry.get(x)[0].id.strike_price)

In [ ]:
# Take any list, plot the option price by strike
def price_strike_from_tenor(expdate,contracts):

    if expdate not in contract_by_expiry: 
        return [],[]
    contracts_list = contract_by_expiry.get(expdate)
    # Initialize TradeBar Dataframe
    trade_bar_df = qb.history(TradeBar, contracts_list, tradingday, nextday,Resolution.DAILY)
    dataset=trade_bar_df.index.to_numpy()
    pricedata=[]
    strikedata=[]
    #print(trade_bar_df.columns)

    for x in dataset:   
    #    print(x[1],trade_bar_df.loc[x].values[0])
        strikedata.append(x[1]-atm)
        pricedata.append(trade_bar_df.loc[x].values[0])
    #print(trade_bar_df.head())
    return (pricedata,strikedata)

In [ ]:
prices,strikes = price_strike_from_tenor(0,contract_by_expiry)
#for i in range(25,len(strikes)):
#    print(prices[i],strikes[i])

#plt.plot(strikes,prices)
plt.scatter(strikes,prices,s=1)
plt.xlabel("Moneyness")
plt.ylabel("Option Price")
plt.title(ticker + " options contracts for expiry "+ expiry.strftime('%d/%m/%Y')+" on " + tradingday.strftime('%d/%m/%Y'))
plt.xticks([-0.1*atm,-0.05*atm,0,0.05*atm,0.1*atm],["-10%","-5%","0%","5%","10%"])
#plt.xticks([round(atm, 2),round(atm+(0.8*dist*atm),2),round(atm-(0.8*dist*atm),2)],[round(atm, 2),round(atm+(0.8*dist*atm),2),round(atm-(0.8*dist*atm),2)])
plt.show

In [ ]:
# Typical Black-Scholes calc

def price_from_iv(iv,optiontype,underlying,ttm,rfr,strike,dividend=0,dayformat="Calendar"):
    if dayformat == "Calendar": ttm_annual = ttm/365
    else: ttm_annual = ttm_annual=ttm/252
    #print("iv=",iv," type=",optiontype," underlying=", underlying, " ttm=",ttm_annual, " rfr=", rfr," strike=",strike," dividend=",dividend)
    discount = math.exp(-rfr*ttm_annual)
    forward_price = underlying/discount
    d1=(math.log(forward_price/strike)+(ttm_annual*iv**2)/2)/iv/math.sqrt(ttm_annual)
    d2=d1-(iv*math.sqrt(ttm_annual))
    #print("discount=",discount," forward price=",forward_price," d1=",d1," d2=",d2)
    callprice = discount*((NormalDist(mu=0, sigma=1).cdf(d1)*forward_price)-(NormalDist(mu=0, sigma=1).cdf(d2)*strike))
    
    if optiontype==1: return round(callprice,6)
    else:
        putprice= callprice-underlying+discount*strike
        return round(putprice,6)

#print(price_from_iv(0.25,1,105,30,0.04,100))


# Implied vol simulation

def iv_from_price(price,optiontype,underlying,ttm,rfr,strike,dividend=0):
    lowguess=0
    highguess=0.5
    check=0
    price=round(price,6)
    while check !=price:
        guess=(lowguess+highguess)/2
        check = round(price_from_iv(guess,optiontype,underlying,ttm,rfr,strike,dividend),6)
        if check<price:
            lowguess=guess
            continue
        if check>price:
            highguess=guess
            continue
    return(round(guess,4))
print(iv_from_price(0.25,1,100,30,0.04,105))

In [ ]:
tenor=0
expiry= list(contract_by_expiry.keys())[tenor]
prices,strikes = price_strike_from_tenor(expiry,contract_by_expiry)
#for i in range(25,len(strikes)):
#    print(prices[i],strikes[i])

#plt.plot(strikes,prices)
plt.scatter(strikes,prices,s=1)
plt.xlabel("Moneyness")
plt.ylabel("Option Price")
plt.title(ticker + " options contracts for expiry "+ expiry.strftime('%d/%m/%Y')+" on " + tradingday.strftime('%d/%m/%Y'))
plt.xticks([-0.1*atm,-0.05*atm,0,0.05*atm,0.1*atm],["-10%","-5%","0%","5%","10%"])
#plt.xticks([round(atm, 2),round(atm+(0.8*dist*atm),2),round(atm-(0.8*dist*atm),2)],[round(atm, 2),round(atm+(0.8*dist*atm),2),round(atm-(0.8*dist*atm),2)])
plt.show

In [ ]:
volsurf_data=[]

for x in range(len(list(contract_by_expiry.keys()))):
    expiry= list(contract_by_expiry.keys())[x]
    prices,strikes = price_strike_from_tenor(expiry,contract_by_expiry)
    for i in range(len(prices)):
        impliedvol=(prices[i],option_type,atm,x,tenyearrate,strikes[i])
        volsurf_data.append([x,prices[i],strikes[i]])
length = max(map(len, volsurf_data))
volsurf=np.array([x+[None]*(length-len(x)) for x in volsurf_data])

print(volsurf[0,2])




In [ ]:
# Plain 3D plot

fig = plt.figure()
ax = plt.axes(projection='3d')
xdata=volsurf[:,0]
ydata=volsurf[:,2]
zdata=volsurf[:,1]
ax.set_xlabel('DTE')
ax.set_ylabel('Moneyness')
#ax.axes.set_ylim3d(bottom=-40, top=40) 
ax.set_zlabel('Option Price')
#ax.axes.set_zlim3d(bottom=-20, top=20) 
ax.set_zlabel(ticker + " options contracts for expiry "+ expiry.strftime('%d/%m/%Y')+" on " + tradingday.strftime('%d/%m/%Y'))
ax.scatter3D(xdata, ydata, zdata, c=zdata)
ax.view_init(elev=30, azim=15, roll=15)